In [5]:
from PIL import Image
from PIL.ExifTags import TAGS

def extract_focal_info(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()

    if exif_data is None:
        raise ValueError("EXIF 데이터가 없습니다.")

    exif = {}
    for tag, value in exif_data.items():
        tag_name = TAGS.get(tag, tag)
        exif[tag_name] = value

    # 1. 초점 거리 (mm)
    f_mm = exif.get("FocalLength")
    if isinstance(f_mm, tuple):
        f_mm = f_mm[0] / f_mm[1]

    # 2. 35mm 환산 초점 거리
    f_35mm = exif.get("FocalLengthIn35mmFilm")

    # 3. 이미지 너비 (픽셀)
    image_width_px = exif.get("ExifImageWidth", image.width)

    if f_mm is None or f_35mm is None or image_width_px is None:
        raise ValueError("EXIF에서 필요한 정보가 부족합니다.")

    # 4. 센서 가로 길이 (mm)
    sensor_width_mm = (f_mm / f_35mm) * 36.0

    # 5. 픽셀 기준 focal length
    f_pixel = (f_mm / sensor_width_mm) * image_width_px

    return {
        "focal_length_mm": f_mm,
        "focal_length_35mm": f_35mm,
        "sensor_width_mm": sensor_width_mm,
        "image_width_px": image_width_px,
        "focal_length_pixel": f_pixel
    }

# 사용 예시
info = extract_focal_info("./image/problem_2/3.jpg")
for k, v in info.items():
    print(f"{k}: {v}")


focal_length_mm: 18.0
focal_length_35mm: 27
sensor_width_mm: 24.0
image_width_px: 2000
focal_length_pixel: 1500.0


In [6]:
image = Image.open("./image/problem_2/3.jpg")
exif_data = image._getexif()
data = {}
for tag, value in exif_data.items():
    tag_name = TAGS.get(tag, tag)
    data[tag_name] = value

print(data.keys())

dict_keys(['PrintImageMatching', 'ResolutionUnit', 'ExifOffset', 'ImageDescription', 'Make', 'Model', 'Software', 'Orientation', 'DateTime', 'YCbCrPositioning', 'XResolution', 'YResolution', 'ExifVersion', 'ComponentsConfiguration', 'CompressedBitsPerPixel', 'DateTimeOriginal', 'DateTimeDigitized', 'BrightnessValue', 'ExposureBiasValue', 'MaxApertureValue', 'MeteringMode', 'LightSource', 'Flash', 'FocalLength', 'UserComment', 'ColorSpace', 'ExifImageWidth', 'SceneCaptureType', 'ExifImageHeight', 'Contrast', 'Saturation', 'Sharpness', 'FileSource', 'ExposureTime', 'ExifInteroperabilityOffset', 'FNumber', 'SceneType', 'ExposureProgram', 'CustomRendered', 'ISOSpeedRatings', 'ExposureMode', 'FlashPixVersion', 'WhiteBalance', 'FocalLengthIn35mmFilm', 'MakerNote'])


In [12]:
print(data['Model'],data['SceneType'],data['FNumber'],data['ExposureTime'],data['ISOSpeedRatings'],data['FocalLength'],data['FocalLengthIn35mmFilm'])

NEX-5N b'\x01' 7.1 0.005 100 18.0 27


In [15]:
def convert_focal_mm_to_pixel(f_mm, sensor_width_mm, image_width_px):
    """
    mm 단위 초점 거리 -> 픽셀 단위 변환
    """
    return (f_mm / sensor_width_mm) * image_width_px


# Sony NEX-5N 기준 값
sensor_width_mm = 23.4
f_mm = data['FocalLength']  # EXIFFocalLength
image_width_px = data['ExifImageWidth']  # EXIFImageWidth

f_pixel = convert_focal_mm_to_pixel(f_mm, sensor_width_mm, image_width_px)
print(f"Focal Length in Pixels: {f_pixel:.2f}")
print(image.size, image_width_px)


Focal Length in Pixels: 1538.46
(2000, 1329) 2000
